In [ ]:
%load_ext itikz

import numpy as np
import matplotlib.pyplot as plt

wavetable synthesis is a sound synthesis technique to create periodic waveforms. it is fundnamentaly based on the periodic reproduction of a single cycle waveform. there is an array with the data of the waveform which is outputed to a DAC.

for the calculation of a single cycle sine waveform we can use this equation:

$$
\begin{array}{c}
y(t) = A\sin(2 \pi f t + \varphi)
\end{array}
$$

where:

A, amplitude, the peak deviation of the function from zero.

f, ordinary frequency, the number of oscillations (cycles) that occur each second of time.

\varphi , phase, specifies (in radians) where in its cycle the oscillation is at t = 0.

When {\displaystyle \varphi }\varphi  is non-zero, the entire waveform appears to be shifted in time by the amount {\displaystyle \varphi }\varphi /ω seconds. A negative value represents a delay, and a positive value represents an advance.



In [ ]:
x = np.linspace(-np.pi, np.pi, 512)
f = 2
r = 0

data1 = np.sin(x)

fig, ax1 = plt.subplots(figsize=(8, 4))

ax1.set_xlabel('t')
ax1.set_ylabel('A')
ax1.plot(data1, color='Red')

plt.tight_layout()
plt.show()


the output is a single cycle waveform with the amplitude from -1 to +1. when this waveform is streamed to the DAC of the STM32 processor the output is the excpected wveform. But what is the frequency? First we need to calculate the playback frequency of the STM32 interrupt. This means how fast the sample points are streamed to the DAC.


$$
\begin{array}{c}
TIM_{CLK}/(TIM_{PSC+1})/(TIM_{ARR + 1})
\end{array}
$$

where:

TIM_{CLK}: The clock of the interrupt

TIM_{PSC}: The Prescaler Value

TIM_{ARR}: the counter


$$
\begin{array}{c}
16MHz/(0+1)/(32 + 1) = 484848Hz
\end{array}
$$


this is the speed the sample point are streamed to the DAC and not the output frequency. The table is one cycle, so the output frequency is the streaming frequency divided by the table length.

## calculate frequency

calculate the playback frequency:
<img src="http://latex.codecogs.com/gif.latex?f%20%3D%20%5Cfrac%7B1%7D%7Bt_%7Bint%7D*size_%7Btable%7D%7D"/><br/>

for increment = 1:

calculate the wavetable on the stm32

```
#define WAVETABLE_SIZE 1024

float32_t wt_sine[WAVETABLE_SIZE];

float32_t start = -3.14;
float32_t step = 2 * 3.14 / WAVETABLE_SIZE;

for( int i=0; i<WAVETABLE_SIZE; ++i ) {
    wt_sine[i] = arm_sin_f32( start );
    start += step;
}

```


In [ ]:
TIMclk = 1.6e+7
TIMpsc = 0
TIMarr = 128

res = TIMclk / (TIMpsc + 1) / (TIMarr + 1)
print(res)

In [ ]:
frequency = res
steps = 512
increment = 1

print("1/((1/40000)*1024) = %f Hz" %(1/((1/frequency)*(steps/increment)))) 

In [ ]:
print( 4194304 >> 22)

In [ ]:
%%itikz --temp-dir
\documentclass[tikz]{standalone}
\usetikzlibrary{arrows}
\begin{document}
\begin{tikzpicture}[scale=3,cap=round]
% Local definitions
\def\costhirty{0.8660256}
% Colors
\colorlet{anglecolor}{green!50!black}
\colorlet{sincolor}{red}
\colorlet{tancolor}{orange!80!black}
\colorlet{coscolor}{blue}
% Styles
\tikzstyle{axes}=[]
\tikzstyle{important line}=[very thick]
\tikzstyle{information text}=[rounded corners,fill=red!10,inner sep=1ex]
% The graphic
\draw[style=help lines,step=0.5cm] (-1.4,-1.4) grid (1.4,1.4);
\draw (0,0) circle (1cm);
\begin{scope}[style=axes]
\draw[->] (-1.5,0) -- (1.5,0) node[right] {$x$} coordinate(x axis);
\draw[->] (0,-1.5) -- (0,1.5) node[above] {$y$} coordinate(y axis);
\foreach \x/\xtext in {-1, -.5/-\frac{1}{2}, 1}
\draw[xshift=\x cm] (0pt,1pt) -- (0pt,-1pt) node[below,fill=white] {$\xtext$};
\foreach \y/\ytext in {-1, -.5/-\frac{1}{2}, .5/\frac{1}{2}, 1}
\draw[yshift=\y cm] (1pt,0pt) -- (-1pt,0pt) node[left,fill=white] {$\ytext$};
\end{scope}
\filldraw[fill=green!20,draw=anglecolor] (0,0) -- (3mm,0pt) arc(0:30:3mm);
\draw (15:2mm) node[anglecolor] {$\alpha$};
\draw[style=important line,sincolor]
(30:1cm) -- node[left=1pt,fill=white] {$\sin \alpha$} (30:1cm |- x axis);
\draw[style=important line,coscolor]
(30:1cm |- x axis) -- node[below=2pt,fill=white] {$\cos \alpha$} (0,0);
\draw[style=important line,tancolor] (1,0) -- node[right=1pt,fill=white] {
$\displaystyle \tan \alpha \color{black}=
\frac{{\color{sincolor}\sin \alpha}}{\color{coscolor}\cos \alpha}$}
(intersection of 0,0--30:1cm and 1,0--1,1) coordinate (t);
\draw (0,0) -- (t);
\draw[xshift=1.85cm]
node[right,text width=6cm,style=information text]
{
The {\color{anglecolor} angle $\alpha$} is $30^\circ$ in the
example ($\pi/6$ in radians). The {\color{sincolor}sine of
$\alpha$}, which is the height of the red line, is
\[
{\color{sincolor} \sin \alpha} = 1/2.
\]
By the Theorem of Pythagoras ...
};
\end{tikzpicture}
\end{document}
